In [1]:
import pandas as pd
import numpy as np
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver # driver de selenium

import time   # para manejar tiempo

In [2]:
from selenium.webdriver.chrome.options import Options   # opciones del driver

from selenium.webdriver.common.by import By   # para buscar segun tag, id, class, etc...

from selenium.webdriver.support.ui import WebDriverWait   # es para esperar

from selenium.webdriver.support import expected_conditions as EC  # condiciones esperadas...

from selenium.webdriver import ActionChains as AC   # acciones encadenadas, rollo doble click

from selenium.webdriver.common.keys import Keys  # manejar teclas

In [3]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

#### 1. Conecto con la página WEB

In [4]:
PATH = 'driver/chromedriver'

driver = webdriver.Chrome()   # abre una ventana de chrome 
                                # una vez que he abierto una ventana, puedo trabajar siempre sobre esta misma.

In [5]:
url = 'https://www.entradas.com/city/madrid-370/conciertos-y-festivales-85/'

driver.get(url)

Conecto con la Web de Entradas.com para trabajar sobre esta, y hacer scraping de los datos que quiero. Utilizaré el método de Scrapeo de selenium.

#### 2. Aceptar las cookies 

- Lo primero que hago es darle a Aceptas las cookies del sitio web. 
- Para ello tengo un problema inicial, ya que está categorizada como una pestaña "shadow" por lo que cuando busco la opción de hacer click al botón de aceptar, aparece como una pestaña / lista vacía. 

In [6]:
div_shadow = driver.find_element(By.XPATH, '//div[@class="cmpwrapper"]')

In [7]:
# Check if the element has a Shadow DOM using JavaScript
has_shadow_dom = driver.execute_script('return arguments[0].shadowRoot !== null', div_shadow)

if has_shadow_dom:
    print("Element has a Shadow DOM")
else:
    print("Element does not have a Shadow DOM")

Element has a Shadow DOM


In [8]:
shadow_root = driver.execute_script('return arguments[0].shadowRoot', div_shadow)

In [9]:
shadow_root.find_elements(By.CSS_SELECTOR, '#cmpwelcomebtnyes > a')[0].click()

- Una vez consigo traspasar la barrera del 'Shadow DOM', entonces puedo comenzar con el scrapeo de las tablas que necesito para mi estudio. 

#### 3. Empezamos el scrapeo por eventos en tabla

In [10]:
tabla = '//product-group-item'

In [12]:
table = driver.find_elements(By.XPATH, tabla)
table

[<selenium.webdriver.remote.webelement.WebElement (session="dda195b765d63930079c4d1195f96f4d", element="AFDB89D35C0745C64C7AD9B4A6B60048_element_34")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dda195b765d63930079c4d1195f96f4d", element="AFDB89D35C0745C64C7AD9B4A6B60048_element_35")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dda195b765d63930079c4d1195f96f4d", element="AFDB89D35C0745C64C7AD9B4A6B60048_element_36")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dda195b765d63930079c4d1195f96f4d", element="AFDB89D35C0745C64C7AD9B4A6B60048_element_37")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dda195b765d63930079c4d1195f96f4d", element="AFDB89D35C0745C64C7AD9B4A6B60048_element_38")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dda195b765d63930079c4d1195f96f4d", element="AFDB89D35C0745C64C7AD9B4A6B60048_element_39")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dda195b765d63930079c4d1195f9

In [13]:
type(table)

list

In [14]:
elementos = driver.find_elements(By.XPATH, '//*[@id="list-item-0"]')
elementos

[<selenium.webdriver.remote.webelement.WebElement (session="dda195b765d63930079c4d1195f96f4d", element="AFDB89D35C0745C64C7AD9B4A6B60048_element_56")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dda195b765d63930079c4d1195f96f4d", element="AFDB89D35C0745C64C7AD9B4A6B60048_element_57")>]

In [15]:
type(elementos)

list

In [16]:
elementos[0].text

'Cariño\nInverfest 2024\n13/01/20241 Evento\n desde 19,80 €\nMostrar 1 Evento'

In [17]:
elementos = elementos[0].text.split('\n')
elementos

['Cariño',
 'Inverfest 2024',
 '13/01/20241 Evento',
 ' desde 19,80 €',
 'Mostrar 1 Evento']

In [18]:
elements = driver.find_elements(By.XPATH, '//div[@class="search-result-content"]')
elements

[<selenium.webdriver.remote.webelement.WebElement (session="dda195b765d63930079c4d1195f96f4d", element="AFDB89D35C0745C64C7AD9B4A6B60048_element_58")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dda195b765d63930079c4d1195f96f4d", element="AFDB89D35C0745C64C7AD9B4A6B60048_element_59")>]

In [31]:
data = []

for element in elements:
    artist_name = element.find_elements(By.XPATH, '//div[@class="event-listing-city theme-text-color"]')
    astists = [artist.text for artist in artist_name]
    date_elements = element.find_elements(By.XPATH, '//div//span[@class="listing-data theme-text-color"]//span')
    dates = [date.text for date in date_elements]
    price = element.find_elements(By.XPATH, '//div/listing-cta/event-status/div/span/span')
    precios = [precio.text for precio in price]
    
    data.append([astists, dates, precios])

for entry in data:
    print(entry)

[['Cariño', 'Iván Ferreiro', 'The World Of Hans Zimmer', 'Triángulo de Amor Bizarro', 'Nico B', 'Cala Vento', 'Sidonie', 'Belako - Sigo regando', 'Biznaga', 'Morreo - Alegría Club!', 'Diego el Cigala - Obras Maestras', 'Symphonic Rhapsody of Queen', 'Sexy Zebras', 'Biografía - Rafa Sánchez, de la unión', 'OBK', 'El Consorcio - Toda una vida', 'Rock en Familia - Hijos del Rock and Roll', 'Rock en Familia - I Love Rock and Roll', 'Rock en Familia - Queenmanía', 'Rock en Familia - I Love Heavy Metal'], ['13/01/2024', '1 Evento', '13/01/2024', '1 Evento', '24/04/2024 ─ 29/04/2024', '2 eventos', '29/12/2023', '1 Evento', '14/01/2024', '1 Evento', '01/12/2023', '1 Evento', '12/01/2024 ─ 14/01/2024', '2 eventos', '20/01/2024', '1 Evento', '26/01/2024', '1 Evento', '18/01/2024', '1 Evento', '11/12/2023 ─ 18/12/2023', '2 eventos', '16/01/2024', '1 Evento', '01/02/2024', '1 Evento', '11/12/2023', '1 Evento', '13/01/2024 ─ 27/01/2024', '3 eventos', '27/11/2023 ─ 28/11/2023', '2 eventos', '05/11/2

In [32]:
type(entry)

list

In [33]:
artistas = entry[0]
artistas

['Cariño',
 'Iván Ferreiro',
 'The World Of Hans Zimmer',
 'Triángulo de Amor Bizarro',
 'Nico B',
 'Cala Vento',
 'Sidonie',
 'Belako - Sigo regando',
 'Biznaga',
 'Morreo - Alegría Club!',
 'Diego el Cigala - Obras Maestras',
 'Symphonic Rhapsody of Queen',
 'Sexy Zebras',
 'Biografía - Rafa Sánchez, de la unión',
 'OBK',
 'El Consorcio - Toda una vida',
 'Rock en Familia - Hijos del Rock and Roll',
 'Rock en Familia - I Love Rock and Roll',
 'Rock en Familia - Queenmanía',
 'Rock en Familia - I Love Heavy Metal']

In [34]:
fecha_eventos = entry[1]
fecha_eventos

['13/01/2024',
 '1 Evento',
 '13/01/2024',
 '1 Evento',
 '24/04/2024 ─ 29/04/2024',
 '2 eventos',
 '29/12/2023',
 '1 Evento',
 '14/01/2024',
 '1 Evento',
 '01/12/2023',
 '1 Evento',
 '12/01/2024 ─ 14/01/2024',
 '2 eventos',
 '20/01/2024',
 '1 Evento',
 '26/01/2024',
 '1 Evento',
 '18/01/2024',
 '1 Evento',
 '11/12/2023 ─ 18/12/2023',
 '2 eventos',
 '16/01/2024',
 '1 Evento',
 '01/02/2024',
 '1 Evento',
 '11/12/2023',
 '1 Evento',
 '13/01/2024 ─ 27/01/2024',
 '3 eventos',
 '27/11/2023 ─ 28/11/2023',
 '2 eventos',
 '05/11/2023 ─ 10/03/2024',
 '2 eventos',
 '03/12/2023',
 '1 Evento',
 '17/12/2023 ─ 14/04/2024',
 '2 eventos',
 '19/11/2023 ─ 24/03/2024',
 '2 eventos']

In [35]:
precios = entry[2]
precios


['desde 19,80 €',
 'desde 38,50 €',
 'desde 55,00 €',
 'desde 19,80 €',
 'desde 13,50 €',
 'desde 19,80 €',
 'desde 27,50 €',
 'desde 17,60 €',
 'desde 18,70 €',
 'desde 14,50 €',
 'desde 55,00 €',
 'desde 35,00 €',
 'desde 18,00 €',
 'desde 35,00 €',
 'desde 30,00 €',
 'desde 29,00 €',
 'desde 7,50 €',
 'desde 7,50 €',
 'desde 7,50 €',
 'desde 7,50 €']

In [36]:
fechas = []
num_eventos = []

for i in range(0, len(fecha_eventos), 2):
    fechas.append(fecha_eventos[i])

for i in range(1, len(fecha_eventos), 2):
    num_eventos.append(fecha_eventos[i])

print(fechas)
print(num_eventos)

['13/01/2024', '13/01/2024', '24/04/2024 ─ 29/04/2024', '29/12/2023', '14/01/2024', '01/12/2023', '12/01/2024 ─ 14/01/2024', '20/01/2024', '26/01/2024', '18/01/2024', '11/12/2023 ─ 18/12/2023', '16/01/2024', '01/02/2024', '11/12/2023', '13/01/2024 ─ 27/01/2024', '27/11/2023 ─ 28/11/2023', '05/11/2023 ─ 10/03/2024', '03/12/2023', '17/12/2023 ─ 14/04/2024', '19/11/2023 ─ 24/03/2024']
['1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '2 eventos', '1 Evento', '1 Evento', '1 Evento', '3 eventos', '2 eventos', '2 eventos', '1 Evento', '2 eventos', '2 eventos']


In [37]:
import pandas as pd

In [38]:
datos = pd.DataFrame({'Artista': artistas, 'Fechas': fechas, 'Eventos': num_eventos, 'Precios': precios})

datos

,Artista,Fechas,Eventos,Precios
0,Cariño,13/01/2024,1 Evento,"desde 19,80 €"
1,Iván Ferreiro,13/01/2024,1 Evento,"desde 38,50 €"
2,The World Of Hans Zimmer,24/04/2024 ─ 29/04/2024,2 eventos,"desde 55,00 €"
3,Triángulo de Amor Bizarro,29/12/2023,1 Evento,"desde 19,80 €"
4,Nico B,14/01/2024,1 Evento,"desde 13,50 €"
5,Cala Vento,01/12/2023,1 Evento,"desde 19,80 €"
6,Sidonie,12/01/2024 ─ 14/01/2024,2 eventos,"desde 27,50 €"
7,Belako - Sigo regando,20/01/2024,1 Evento,"desde 17,60 €"
8,Biznaga,26/01/2024,1 Evento,"desde 18,70 €"
9,Morreo - Alegría Club!,18/01/2024,1 Evento,"desde 14,50 €"
